In [1]:
%%capture
!pip install "datasets==3.6.0" -q
!git clone https://github.com/huutrank4ds/vi-tacotron2.git -q
# !git clone --single-branch --branch develop https://github.com/huutrank4ds/vi-tacotron2.git -q

import sys
sys.path.append('/kaggle/working/vi-tacotron2')

In [2]:
# sys.path.remove('/kaggle/working/vi-tacotron2')
# !rm -rf vi-tacotron2

In [3]:
# import os
# import shutil

# src = '/kaggle/input/final-vi-tacotron2/pytorch/default/1'
# dst = '/kaggle/working/checkpoints'

# os.makedirs(dst, exist_ok=True)

# # Danh sách 1: best, last và file csv
# list_keep = []

# # Danh sách 2: checkpoint theo epoch
# list_epoch = []

# for fname in os.listdir(src):
#     src_path = os.path.join(src, fname)

#     if not os.path.isfile(src_path):
#         continue

#     # Nhóm 1
#     if (
#         fname in [
#             'checkpoint_vi_tacotron2_best.pt',
#             'checkpoint_vi_tacotron2_last.pt'
#         ]
#         or fname.endswith('.csv')
#     ):
#         list_keep.append(fname)

#     # Nhóm 2
#     elif fname.startswith('checkpoint_vi_tacotron2_epoch_') and fname.endswith('.pt'):
#         list_epoch.append(fname)

# # Copy cả 2 danh sách
# for fname in list_keep:
#     shutil.copy(
#         os.path.join(src, fname),
#         os.path.join(dst, fname)
#     )

# print(f'Copied {len(list_keep)} keep-files')
# # print(f'Copied {len(list_epoch)} epoch-files')

In [4]:
# from huggingface_hub import login

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# # login với secret
# login(token=user_secrets.get_secret("HF_TOKEN"))

In [ ]:
import torch
import torch.multiprocessing as mp
from workers import train_worker_chunk_by_chunk
from functools import partial
from config import Hparams
import os
import json


hparams = Hparams()
hparams.parquet_valid_file = '/kaggle/input/phoaudiobook-validation/validation-00000-of-00001.parquet'
hparams.batch_size = 224
# hparams.val_interval = 1000
# hparams.max_step_training = 8000
hparams.name_file_checkpoint = 'checkpoint_vi_tacotron2_last.pt'
hparams.speaker_embeddings_file = '/kaggle/input/mean-speaker-embeddings-phoaudiobook/mean_speaker_embeddings.pt'
hparams.validation_speaker_embeddings_file = '/kaggle/input/mean-speaker-embeddings-phoaudiobook/mean_speaker_embeddings_validation.pt'
hparams.learning_rate = 1e-3
hparams.n_frames_per_step = 2
hparams.fp16_run = False
hparams.guided_attention = True
hparams.max_epochs = 15
hparams.dataset_chunks = [f'/kaggle/input/phoaudiobook-{i}' for i in range(8, 13)]

metadata_path = '/kaggle/input/phoaudiobook-parquet-metadata/phoaudiobook_parquet_metadata.json'
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

map_to_total = {int(x['name_folder'].split('-')[1]): x['total_records'] for x in metadata}
hparams.metadata = map_to_total

if __name__ == "__main__":    
    world_size = torch.cuda.device_count()
    
    # Cách dùng partial là rất tốt, giữ nguyên
    worker_func = partial(train_worker_chunk_by_chunk, hparams=hparams)
    
    if world_size > 1:
        print(f"Đã phát hiện {world_size} GPUs. Bắt đầu DDP Training (mp.spawn)...")
        
        # QUAN TRỌNG: Bật cờ DDP lên
        hparams.ddp_run = True 
        
        mp.spawn(worker_func,
                 args=(world_size,),
                 nprocs=world_size,
                 join=True)
    else:
        print("Chỉ tìm thấy 1 GPU (hoặc CPU). Chạy chế độ Single-GPU (Normal mode).")
        
        # QUAN TRỌNG: Tắt cờ DDP đi
        hparams.ddp_run = False
        
        # Gọi hàm worker trực tiếp (không qua spawn)
        # Rank luôn là 0, World_size là 1
        train_worker_chunk_by_chunk(0, 1, hparams)

Chỉ tìm thấy 1 GPU (hoặc CPU). Chạy chế độ Single-GPU (Normal mode).
[Rank 0] Guided Attention is ENABLED with sigma=0.2.
[Rank 0] Not using GradScaler.
[Rank 0] Preparing Validation Set...


Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]


============================== Epoch 0 ==============================
[Rank 0] Generating cache for chunk 3...


Resolving data files:   0%|          | 0/141 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/1 [00:00<?, ?batch/s]


[Rank 0] Epoch 0 | Chunk 0 | Train Loss: 3.93766, Train_Mel: 1.91349, Train_Post: 1.84257, Train_Gate: 0.18046 
| Val Loss: 2.45811 | Mel: 1.27470 | Post: 1.12511 | Gate: 0.05830
Saved checkpoint at checkpoints/checkpoint_vi_tacotron2_best.pt
Saved NEW BEST model: checkpoint_vi_tacotron2_best.pt
[Rank 0] Generating cache for chunk 1...


Resolving data files:   0%|          | 0/139 [00:00<?, ?it/s]

[Rank 0] Dataset chunk 1 ready.
[Rank 0] DataLoader for chunk 1 created.


Epoch 0| Chunk 1 (Real: phoaudiobook-9):   0%|          | 0/180 [00:00<?, ?batch/s]